## 使用素数群（请启动 SageMath Notebook 服务）

In [2]:
import random    # 随机数生成器
import secrets   # 随机数生成器
import sys

class ElGamalFixedParams:
    def __init__(self, p = 2**61 - 1, g = 5):
        """初始化ElGamal加密系统，使用预定义的素数乘法群"""
        self.p = p
        self.g = g
        self.q = self.p - 1
        self.is_sage = 'sage' in sys.modules
        
    def keygen(self):
        """生成公钥和私钥"""
        if self.is_sage:
            x = ZZ.random_element(1, self.q)
        else:
            x = secrets.randbelow(self.q)
            
        h = pow(self.g, x, self.p)
        public_key = (self.p, self.g, h)
        private_key = (self.p, self.g, x)
        return public_key, private_key
    
    def encrypt(self, public_key, message):
        """加密消息m∈G"""
        p, g, h = public_key
        if message >= p:
            raise ValueError("消息必须小于群的阶")
        if self.is_sage:
            y = ZZ.random_element(1, self.q)
        else:
            y = secrets.randbelow(self.q)

        c1 = pow(g, y, p)
        c2 = (pow(h, y, p) * message) % p
        return (c1, c2)
    
    def decrypt(self, private_key, ciphertext):
        """解密密文"""
        p, g, x = private_key
        c1, c2 = ciphertext
        c1_x = pow(c1, x, p)
        c1_x_inv = pow(c1_x, p-2, p)  
        plaintext = (c2 * c1_x_inv) % p
        
        return plaintext

In [2]:
elgamal = ElGamalFixedParams()
public_key, private_key = elgamal.keygen()

# 消息必须是群G中的元素
message = ZZ.random_element(1, elgamal.q) 
ciphertext = elgamal.encrypt(public_key, message)

decrypted_message = elgamal.decrypt(private_key, ciphertext)
    
print(f"原始消息: {message}")
print(f"密文: {ciphertext}")
print(f"解密后消息: {decrypted_message}")
print(f"解密是否成功: {message == decrypted_message}")

原始消息: 1602220284556854402
密文: (416367735494003930, 2099606449962504160)
解密后消息: 1602220284556854402
解密是否成功: True


In [3]:
fp = GF(0xEEFC0B79D5FF2502BA4BC0C1BF86293C1B0495086E25C075C1391EC8DD3B1961)
elgamal = ElGamalFixedParams(0xEEFC0B79D5FF2502BA4BC0C1BF86293C1B0495086E25C075C1391EC8DD3B1961, fp.multiplicative_generator())
public_key, private_key = elgamal.keygen()

# 消息必须是群G中的元素
message = ZZ.random_element(1, elgamal.q) 
ciphertext = elgamal.encrypt(public_key, message)

decrypted_message = elgamal.decrypt(private_key, ciphertext)
    
print(f"原始消息: {message}")
print(f"密文: {ciphertext}")
print(f"解密后消息: {decrypted_message}")
print(f"解密是否成功: {message == decrypted_message}")

原始消息: 16975800156874262171609312747572765507776701200048189872396529461226036955749
密文: (94101740137505956740809636677320850957420484158275552592777235244496754535588, 38904294573610380306958764000303608302278367061812480223508399143507642063332)
解密后消息: 16975800156874262171609312747572765507776701200048189872396529461226036955749
解密是否成功: True


## 使用椭圆曲线群（请启动 Python Jupyter 服务）

1. 使用 Jacobi 投影坐标系

In [1]:
import random
import os

class ElGamalEllipticCurve_Jacobi:
    @classmethod
    def mod_inverse(cls, a, m):
        """计算模逆元 a^-1 mod m"""
        if a == 0:
            return 0
        lm, hm = 1, 0
        low, high = a % m, m
        while low > 1:
            r = high // low
            nm, new = hm - lm * r, high - low * r
            lm, low, hm, high = nm, new, lm, low
        return lm % m
    
    @classmethod
    def mod_pow(cls, base, exponent, modulus):
        """计算模幂 base^exponent mod modulus"""
        if modulus == 1:
            return 0
        result = 1
        base = base % modulus
        while exponent > 0:
            if exponent % 2 == 1:
                result = (result * base) % modulus
            exponent = exponent >> 1
            base = (base * base) % modulus
        return result
        
    def __init__(self, help = False,
                 INF = (0, 1, 0),
                 p:int = 0xFFFFFFFF00000001000000000000000000000000FFFFFFFFFFFFFFFFFFFFFFFF, 
                 a:int = 0xFFFFFFFF00000001000000000000000000000000FFFFFFFFFFFFFFFFFFFFFFFC, 
                 b:int = 0x5AC635D8AA3A93E7B3EBBD55769886BC651D06B0CC53B0F63BCE3C3E27D2604B, 
                 G:tuple[int, int, int] = (0x6B17D1F2E12C4247F8BCE6E563A440F277037D812DEB33A0F4A13945D898C296, 0x4FE342E2FE1A7F9B8EE7EB4A7C0F9E162BCE33576B315ECECBB6406837BF51F5, 1), 
                 n:int = 0xFFFFFFFF00000000FFFFFFFFFFFFFFFFBCE6FAADA7179E84F3B9CAC2FC632551):
        """初始化ElGamal加密系统，使用椭圆曲线群"""
        if help:
            print("""default value :
                 INF = (0, 1, 0),
                 p:int = 0xFFFFFFFF00000001000000000000000000000000FFFFFFFFFFFFFFFFFFFFFFFF, 
                 a:int = 0xFFFFFFFF00000001000000000000000000000000FFFFFFFFFFFFFFFFFFFFFFFC, 
                 b:int = 0x5AC635D8AA3A93E7B3EBBD55769886BC651D06B0CC53B0F63BCE3C3E27D2604B, 
                 G:tuple[int, int, int] = (0x6B17D1F2E12C4247F8BCE6E563A440F277037D812DEB33A0F4A13945D898C296, 0x4FE342E2FE1A7F9B8EE7EB4A7C0F9E162BCE33576B315ECECBB6406837BF51F5, 1), 
                 n:int = 0xFFFFFFFF00000000FFFFFFFFFFFFFFFFBCE6FAADA7179E84F3B9CAC2FC632551""")
        self.INF = INF
        self.p = p
        self.a = a
        self.b = b
        self.G = G
        self.n = n

    def point_add_jacobian(self, P, Q):
        """雅可比坐标系下的点加法 P + Q"""
        if P == self.INF:
            return Q
        if Q == self.INF:
            return P
        X1, Y1, Z1 = P
        X2, Y2, Z2 = Q
        if (X1 * self.mod_pow(Z2, 2, self.p) - X2 * self.mod_pow(Z1, 2, self.p)) % self.p == 0:
            if (Y1 * self.mod_pow(Z2, 3, self.p) + Y2 * self.mod_pow(Z1, 3, self.p)) % self.p == 0:
                return self.INF
        Z1Z1 = (Z1 * Z1) % self.p
        Z2Z2 = (Z2 * Z2) % self.p
        U1 = (X1 * Z2Z2) % self.p
        U2 = (X2 * Z1Z1) % self.p
        S1 = (Y1 * Z2 * Z2Z2) % self.p
        S2 = (Y2 * Z1 * Z1Z1) % self.p
        if U1 == U2:
            if S1 != S2:
                return self.INF
            return self.point_double_jacobian(P)  
        H = (U2 - U1) % self.p
        R = (S2 - S1) % self.p
        H2 = (H * H) % self.p
        H3 = (H * H2) % self.p
        V = (U1 * H2) % self.p
        X3 = (R * R - H3 - 2 * V) % self.p
        Y3 = (R * (V - X3) - S1 * H3) % self.p
        Z3 = (Z1 * Z2 * H) % self.p
        return (X3, Y3, Z3)

    def point_double_jacobian(self, P):
        """雅可比坐标系下的点倍乘 2P"""
        if P == self.INF:
            return self.INF
        X, Y, Z = P
        if Y == 0:
            return self.INF
        Y2 = (Y * Y) % self.p
        Y4 = (Y2 * Y2) % self.p
        S = (4 * X * Y2) % self.p
        M = (3 * X * X + self.a * Z * Z * Z * Z) % self.p
        X3 = (M * M - 2 * S) % self.p
        Y3 = (M * (S - X3) - 8 * Y4) % self.p
        Z3 = (2 * Y * Z) % self.p
        return (X3, Y3, Z3)

    def point_mul_jacobian(self, P, k):
        """雅可比坐标系下的点乘法 kP"""
        if k == 0:
            return self.INF
        result = self.INF
        addend = P
        while k > 0:
            if k & 1:
                result = self.point_add_jacobian(result, addend)
            addend = self.point_double_jacobian(addend)
            k >>= 1
        return result

    def jacobian_to_affine(self, P):
        """将雅可比坐标转换为仿射坐标"""
        if P == self.INF:
            return None  
        X, Y, Z = P
        Z_inv = self.mod_inverse(Z, self.p)
        Z2_inv = (Z_inv * Z_inv) % self.p
        Z3_inv = (Z2_inv * Z_inv) % self.p
        x = (X * Z2_inv) % self.p
        y = (Y * Z3_inv) % self.p
        return (x, y)

    def affine_to_jacobian(self, P):
        """将仿射坐标转换为雅可比坐标"""
        if P is None:
            return self.INF
        x, y = P
        return (x, y, 1)

    def gen_key(self):
        """生成公钥和私钥"""
        x = int.from_bytes(os.urandom(32), 'big') % (self.n-1) + 1
        h = self.point_mul_jacobian(self.G, x)
        public_key = (h,)
        private_key = (x,)
        return public_key, private_key

    def encrypt(self, pk, m_point):
        """加密椭圆曲线上的点消息"""
        h = pk[0]
        m_jacobian = self.affine_to_jacobian(m_point)
    
        y = int.from_bytes(os.urandom(32), 'big') % (self.n-1) + 1
        c1 = self.point_mul_jacobian(self.G, y)
        shared_secret = self.point_mul_jacobian(h, y)
        c2 = self.point_add_jacobian(m_jacobian, shared_secret)
    
        return (c1, c2)

    def decrypt(self, sk, ciphertext):
        """解密密文"""
        x = sk[0]
        c1, c2 = ciphertext
        shared_secret = self.point_mul_jacobian(c1, x)
        shared_secret_neg = (shared_secret[0], (-shared_secret[1]) % self.p, shared_secret[2])
        m_jacobian = self.point_add_jacobian(c2, shared_secret_neg)
        return self.jacobian_to_affine(m_jacobian)

    def point_to_bytes(self, point):
        """将椭圆曲线点转换为字节（未压缩格式）"""
        if point is None:
            return b'\x00'
    
        x, y = point
        point_len = (self.p.bit_length() + 7) // 8
        x_bytes = x.to_bytes(point_len, 'big')
        y_bytes = y.to_bytes(point_len, 'big')
        return b'\x04' + x_bytes + y_bytes

    def bytes_to_point(self, data):
        """将字节转换为椭圆曲线点"""
        if data[0] == 0x00:
            return None
    
        if data[0] != 0x04:
            raise ValueError("仅支持未压缩点格式")
    
        point_len = (self.p.bit_length() + 7) // 8
        x = int.from_bytes(data[1:1+point_len], 'big')
        y = int.from_bytes(data[1+point_len:1+2*point_len], 'big')
        left = (y * y) % self.p
        right = (x * x * x + self.a * x + self.b) % self.p
    
        if left != right:
            raise ValueError("点不在曲线上")
    
        return (x, y)

    def is_on_curve(self, point):
        """检查点是否在曲线上"""
        if point is None:
            return False
    
        x, y = point
        left = (y * y) % self.p
        right = (x * x * x + self.a * x + self.b) % self.p
        return left == right
    
    def random_test(self, times = 1):
        """随机测试加密和解密"""
        for _ in range(times):
            print("=== 自定义椭圆曲线上的ElGamal加密演示 ===")
            G_affine = self.jacobian_to_affine(self.G)
            print(f"验证基点是否在曲线上: {'✅' if self.is_on_curve(G_affine) else '❌'}")

            public_key, private_key = self.gen_key()
            print("✅ 密钥对已生成")
            print(f"私钥 x: {private_key[0]}")
            h_affine = self.jacobian_to_affine(public_key[0])
            print(f"公钥 h: {self.point_to_bytes(h_affine).hex()}")

            r = random.randint(1, self.n-1)
            m_point = self.jacobian_to_affine(self.point_mul_jacobian(self.G, r))
            print("\n🔒 生成随机消息点:")
            print(f"消息点 m: {self.point_to_bytes(m_point).hex()}")
            print(f"坐标 x: {hex(m_point[0])}")
            print(f"坐标 y: {hex(m_point[1])}")

            ciphertext = self.encrypt(public_key, m_point)
            print("\n🔐 加密完成")
            c1_affine = self.jacobian_to_affine(ciphertext[0])
            c2_affine = self.jacobian_to_affine(ciphertext[1])
            c1_bytes = self.point_to_bytes(c1_affine)
            c2_bytes = self.point_to_bytes(c2_affine)
            print(f"密文 c1: {c1_bytes.hex()}")
            print(f"密文 c2: {c2_bytes.hex()}")

            try:
                c1_point = self.bytes_to_point(c1_bytes)
                c2_point = self.bytes_to_point(c2_bytes)
                c1_jacobian = self.affine_to_jacobian(c1_point)
                c2_jacobian = self.affine_to_jacobian(c2_point)
        
                decrypted_point = self.decrypt(private_key, (c1_jacobian, c2_jacobian))
                print("\n🔓 解密完成")
                print(f"解密消息点 m': {self.point_to_bytes(decrypted_point).hex()}")
                print(f"坐标 x: {hex(decrypted_point[0])}")
                print(f"坐标 y: {hex(decrypted_point[1])}")
            except Exception as e:
                print(f"解密错误: {str(e)}")
                exit()
    
            assert m_point[0] == decrypted_point[0] and m_point[1] == decrypted_point[1], "❌ 解密失败! 消息点不匹配"
            print("\n✅ 解密成功！原始消息点与解密消息点完全匹配")

In [2]:
Eg = ElGamalEllipticCurve_Jacobi()
Eg.random_test()

=== 自定义椭圆曲线上的ElGamal加密演示 ===
验证基点是否在曲线上: ✅
✅ 密钥对已生成
私钥 x: 68057127777111319553107925284227018940311392301520392432595151616702285342371
公钥 h: 04ab2df72c89fcd7ec5f4f0370c7353fa43f6436a5b40f059b90e316e1f4e6fc3b0e92914f92e888d996b4a1b88269fbb9ff155dd3ee00afb9cd2f02ddce53b33f

🔒 生成随机消息点:
消息点 m: 041a7fbdd6d175942770481cdffb4ec8301deb33ce38ec90418f005677bc9c54faba9d8999956952d1bd915a7b5eb586751ed7b2b7d9306c34595bd36955f98e4b
坐标 x: 0x1a7fbdd6d175942770481cdffb4ec8301deb33ce38ec90418f005677bc9c54fa
坐标 y: 0xba9d8999956952d1bd915a7b5eb586751ed7b2b7d9306c34595bd36955f98e4b

🔐 加密完成
密文 c1: 0413c91a532ab52728edc2edc441609ea361904ee81d5e6f0cae02f76f716b78dccbccc2620b5d046e07295520e9816a8be5f2f4265a00f9faaaee54504324eded
密文 c2: 0492dc3ef391c6a536bec2c1d66cb3362edd7f6546fd4fa20486a317da2cf51a8dbda0cb4781b734c0ea757d517fbdadba17d5a0f715b782878a044299b6361eb0

🔓 解密完成
解密消息点 m': 041a7fbdd6d175942770481cdffb4ec8301deb33ce38ec90418f005677bc9c54faba9d8999956952d1bd915a7b5eb586751ed7b2b7d9306c34595bd36955f98e4

2. 标准库实现

In [3]:
import os
import random
from ecdsa import NIST256p, ellipticcurve

class ElGamalEllipticCurve_std:
    """基于标准ECDSA的椭圆曲线ElGamal加密系统"""
    
    def __init__(self):
        """初始化椭圆曲线参数"""
        self.curve = NIST256p.curve
        self.G = NIST256p.generator
        self.q = NIST256p.order
        self.p = self.curve.p()
        
    def gen_key(self):
        """生成公钥和私钥"""
        x = int.from_bytes(os.urandom(32), 'big') % (self.q-1) + 1
        h = x * self.G 
        public_key = (self.curve, self.q, self.G, h)
        private_key = (self.curve, self.q, self.G, x)
        return public_key, private_key
    
    def encrypt(self, pk, m_point):
        """加密椭圆曲线上的点消息"""
        _, _, G, h = pk
        y = int.from_bytes(os.urandom(32), 'big') % (self.q-1) + 1
        c1 = y * G            
        shared_secret = y * h   
        c2 = shared_secret + m_point
        return (c1, c2)
    
    def decrypt(self, sk, ciphertext):
        """解密密文"""
        _, _, _, x = sk
        c1, c2 = ciphertext
        shared_secret = x * c1 
        m_point = c2 + (-shared_secret)
        return m_point
    
    def point_to_bytes(self, point):
        """将椭圆曲线点转换为字节（未压缩格式）"""
        point_len = (self.p.bit_length() + 7) // 8
        x_bytes = point.x().to_bytes(point_len, 'big')
        y_bytes = point.y().to_bytes(point_len, 'big')
        return b'\x04' + x_bytes + y_bytes
    
    def bytes_to_point(self, data):
        """将字节转换为椭圆曲线点"""
        if data[0] != 0x04:
            raise ValueError("仅支持未压缩点格式")
        point_len = (self.p.bit_length() + 7) // 8
        x = int.from_bytes(data[1:1+point_len], 'big')
        y = int.from_bytes(data[1+point_len:1+2*point_len], 'big')
        return ellipticcurve.Point(self.curve, x, y, self.q)
    
    def is_on_curve(self, point):
        """检查点是否在曲线上"""
        x = point.x()
        y = point.y()
        left = (y * y) % self.p
        right = (x * x * x + self.curve.a() * x + self.curve.b()) % self.p
        return left == right
    
    def random_test(self, times=1):
        """随机测试加密和解密"""
        for _ in range(times):
            print("=== P-256 椭圆曲线上的ElGamal加密演示 ===")
            
            public_key, private_key = self.gen_key()
            print("✅ 密钥对已生成")
            print(f"私钥 x: {private_key[3]}")
            print(f"公钥 h: {self.point_to_bytes(public_key[3]).hex()}")
            
            r = random.randint(1, self.q-1)
            m_point = r * self.G
            print("\n🔒 生成随机消息点:")
            print(f"消息点 m: {self.point_to_bytes(m_point).hex()}")
            print(f"坐标 x: {hex(m_point.x())}")
            print(f"坐标 y: {hex(m_point.y())}")
            
            ciphertext = self.encrypt(public_key, m_point)
            print("\n🔐 加密完成")
            c1_bytes = self.point_to_bytes(ciphertext[0])
            c2_bytes = self.point_to_bytes(ciphertext[1])
            print(f"密文 c1: {c1_bytes.hex()}")
            print(f"密文 c2: {c2_bytes.hex()}")
            
            try:
                c1_point = self.bytes_to_point(c1_bytes)
                c2_point = self.bytes_to_point(c2_bytes)
                decrypted_point = self.decrypt(private_key, (c1_point, c2_point))
                print("\n🔓 解密完成")
                print(f"解密消息点 m': {self.point_to_bytes(decrypted_point).hex()}")
                print(f"坐标 x: {hex(decrypted_point.x())}")
                print(f"坐标 y: {hex(decrypted_point.y())}")
            except Exception as e:
                print(f"解密错误: {str(e)}")
                exit()
            
            assert m_point.x() == decrypted_point.x() and m_point.y() == decrypted_point.y(), "❌ 解密失败! 消息点不匹配"
            print("\n✅ 解密成功！原始消息点与解密消息点完全匹配")

In [4]:
eg = ElGamalEllipticCurve_std()
eg.random_test()

=== P-256 椭圆曲线上的ElGamal加密演示 ===
✅ 密钥对已生成
私钥 x: 24325669977214531600399449746504830786242208578236652718154075873262224569844
公钥 h: 04db8c4b8a0e50ef229a8e50557fed16a39fb62bd8c07a7cc25749554dd39b1ab0da114034b965ba9564d09e859a3a0e67282e06e584009112e8c881e4067eb2e3

🔒 生成随机消息点:
消息点 m: 0451a281440632a969a181e12e6c43b99a4cce963774e8f4e450da5a7a73204e74dcbaee08f886aca3a9fc1438bdd8686ff001b7f01d075034e76608df9685b7ff
坐标 x: 0x51a281440632a969a181e12e6c43b99a4cce963774e8f4e450da5a7a73204e74
坐标 y: 0xdcbaee08f886aca3a9fc1438bdd8686ff001b7f01d075034e76608df9685b7ff

🔐 加密完成
密文 c1: 0436f592459c34d2a27502b1c509159b6e17ba43a548e7ef220a3e7533e3633cf3cc1649b9a476bf046e23c3615bc023304532f1335e49a27ccafa3e31c9f83e6b
密文 c2: 04207402622b4acde4763baba4d43359a19e4031d60eeb450bccd4a664452e3a4211d7db6f82963b646d0f95af661c6e6db8d4d9be5c7fa2be8d52df44612e0dc1

🔓 解密完成
解密消息点 m': 0451a281440632a969a181e12e6c43b99a4cce963774e8f4e450da5a7a73204e74dcbaee08f886aca3a9fc1438bdd8686ff001b7f01d075034e76608df9685b7ff
坐标 x: 0x5

# 测试

In [7]:
import sys

class ElGamalFixedParams_:
    """对 ElGamalFixedParams 类进行扩展，支持设置 x ,y 参数"""
    def __init__(self, p = 2**61 - 1, g = 5):
        """初始化ElGamal加密系统，使用预定义的素数乘法群"""
        self.p = p
        self.g = g
        self.q = self.p - 1
        self.is_sage = 'sage' in sys.modules
        
    def keygen(self, x_value=None):
        """生成公钥和私钥，可以指定私钥值"""
        if x_value is not None:
            x = x_value % self.q  # 确保x在合法范围内
        else:
            if self.is_sage:
                x = ZZ.random_element(1, self.q)
            else:
                x = secrets.randbelow(self.q)
                
        h = pow(self.g, x, self.p)
        public_key = (self.p, self.g, h)
        private_key = (self.p, self.g, x)
        return public_key, private_key
    
    def encrypt(self, public_key, message, y_value=None):
        """加密消息m∈G，可以指定随机指数y"""
        p, g, h = public_key
        if message >= p:
            raise ValueError("消息必须小于群的阶")
            
        if y_value is not None:
            y = y_value % self.q  # 确保y在合法范围内
        else:
            if self.is_sage:
                y = ZZ.random_element(1, self.q)
            else:
                y = secrets.randbelow(self.q)

        c1 = pow(g, y, p)
        c2 = (pow(h, y, p) * message) % p
        return (c1, c2)
    
    def decrypt(self, private_key, ciphertext):
        """解密密文"""
        p, g, x = private_key
        c1, c2 = ciphertext
        c1_x = pow(c1, x, p)
        c1_x_inv = pow(c1_x, p-2, p)  
        plaintext = (c2 * c1_x_inv) % p
        
        return plaintext

elgamal = ElGamalFixedParams_(0xEEFC0B79D5FF2502BA4BC0C1BF86293C1B0495086E25C075C1391EC8DD3B1961, 5)

x = 65537
public_key, private_key = elgamal.keygen(x)
y = 2**31
message = 2**31 - 1
ciphertext = elgamal.encrypt(public_key, message, y)
decrypted_message = elgamal.decrypt(private_key, ciphertext)
print(f"原始消息: {message}")
print(f"密文: {ciphertext}")
print(f"解密后消息: {decrypted_message}")
print(f"解密是否成功: {message == decrypted_message}")

原始消息: 2147483647
密文: (23590117750720990950683711422750771970328801003254806221129660737059680197796, 41672760796407998237682016831812072015849610929796201122094639249311114013813)
解密后消息: 2147483647
解密是否成功: True
